In [2]:
import parabellum as pb
from jax import random

# create the environment
kwargs = dict(map_width=256, map_height=256, num_allies=10, num_enemies=10)
env = pb.Parabellum(**kwargs)  # <- Parabellum is the central class of parabellum

# initiate stochasticity
rng = random.PRNGKey(0)
rng, key = random.split(rng)

# initialize the environment state
obs, state = env.reset(key)
state_sequence = []

for _ in range(1000):
    # manage stochasticity
    rng, rng_act, key_step = random.split(key, 3)
    key_act = random.split(rng_act, len(env.agents))

    # sample actions and append to state sequence
    act = {a: env.action_space(a).sample(k) for a, k in zip(env.agents, key_act)}

    # step the environment
    state_sequence.append((key_act, state, act))
    obs, state, reward, done, info = env.step(key_step, state, act)


# save visualization of the state sequence
vis = pb.Visualizer(env, state_sequence)  # <- Visualizer is a nice to have class
vis.animate()

ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.